<a href="https://colab.research.google.com/github/mallibus/Unige-DL2019/blob/master/Unige_DL2019_Copy_of_Test_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 0. Introduction to Deep Learning

## 0.1 Basics of TensorFlow
<img src="http://mlclass.epizy.com/lab0_images_notebook/tf_logo.png" width="200px"><br>
TensorFlow is an open source machine learning library for research and production.<br>
It offers tools, libraries and resources that makes it easy for you to build and deploy ML models.

### Most important features:
* Easy model building<br>
Build and train ML models easily using intuitive high-level APIs like Keras with eager execution, which makes for immediate model iteration and easy debugging.


* Robust ML production anywhere<br>
Easily train and deploy models in the cloud, on-prem, in the browser, or on-device no matter what language you use.


* Powerful experimentation for research<br>
A simple and flexible architecture to take new ideas from concept to code, to state-of-the-art models, and to publication faster.


### Import TensorFlow
In this tutorial is used **version 1.13.1** that is the last stable version released.<br>
Recently version 2.0 is released but is still a preview.

In [0]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.python.client import device_lib
import time, h5py, os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib inline

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

### Data representation for neural network
A Tensor consists of a set of primitive values shaped into an array of any number of dimensions.<br>
Similar to NumPy ndarray objects, Tensor objects have a data type and a shape.  <br>
TensorFlow offers a rich library of operations (tf.add, tf.matmul, tf.linalg.inv etc.) that consume and produce Tensors; these operations automatically convert native Python types.<br>
In addition, Tensors can be backed by accelerator memory (like GPU, TPU) and are immutable.<br>

The rank of a tf.Tensor object defines its number of dimensions.

In [0]:
#RANK 0 (scalar)
#Create tensor of type string with value "hello world!" 
#Create tensor of type int16 with value 17
#Create tensor of type float64 with value 3.14159265359
#Create tensor of type complex64 with value 10.1 - 1.5j

t01 = tf.Variable("hello world!", tf.string)
print("RANK 0\n",t01,"\n")


#RANK 1 (vector)
#Create tensor of type string with value "abba" 
#Create tensor of type float32 with value [6.14, 3.001]
#Create tensor of type int32 with value [1,3,5,7]
#Create tensor of type complex64 with value [10.3 - 4.05j, 3.1 - 2.13j]

t11 = tf.Variable(["abba"], tf.string)
print("RANK 1\n",t11,"\n")


#RANK 2 (matrix)
#Create tensor of type int16 with values [7,4] in the first row and [11,1] in the second row 
#Create tensor of type bool with values [False,True] in the first row and [True,False] in the second row 
#Create tensor of type int32 with value [3] in the first row,[2] in the second row,[12] in the third row,[4] in the fourth row
#Create tensor of type float64 with values [0.1,11.2,4.01,3.5] in the first row and [0.2,1,22.1,3.12] in the second row 

t21 = tf.Variable([[7,4],[11,1]], tf.int16)
print("RANK 2\n",t21,"\n")


### Graphical Processing Unit (GPU)

The basic architecture of a GPU differs a lot from a CPU; the GPU is optimized for a high computational power and a high throughput.

<img src="http://mlclass.epizy.com/lab0_images_notebook/cpu_gpu.png" width="500px"><br>

The computation of DNNs is a task that fits excellent on a GPU: there is a large amount of parallelism that can
be utilized (the most common kernels are matrix multiply, convolution and functions with no data dependencies at
all).<br>
In TensorFlow, the supported device types are CPU and GPU; you could use also multiple-GPUs.

In [0]:
if tf.test.is_gpu_available():
    print("GPU ",tf.test.gpu_device_name(), " is available\n")
    
# Print list of available devices
print("Devices available:\n\n",device_lib.list_local_devices())

### Comparison of CPU time and GPU time 
Below there's an example of the computational time required in the different cases to do matrix multiplication.<br>
**What do you expect? Which is faster? Why?**

In [0]:
def measure(x, steps):
    tf.matmul(x, x)
    start = time.time()
    for i in range(steps):
        x = tf.matmul(x, x)
          # tf.matmul can return before completing the matrix multiplication
          # (e.g., can return after enqueing the operation on a CUDA stream).
          # The x.numpy() call below will ensure that all enqueued operations have completed 
          # (and will also copy the result to host memory, so we're including a little more than 
          # just the matmul operation time).
    _ = x.numpy()
    end = time.time()
    return end - start

In [0]:
shape = (1000, 1000)
steps = 200

print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

# Run on CPU:
with tf.device("/cpu:0"):
    print("CPU: {} secs".format(measure(tf.random_normal(shape), steps)))

# Run on GPU, if available:
if tf.test.is_gpu_available():
    with tf.device("/gpu:0"):
        print("GPU: {} secs".format(measure(tf.random_normal(shape), steps)))
else:
    print("GPU: not found")

### Automatic Differentiation
Automatic differentiation is a technique for optimizing machine learning models.<br>
On simple terms, it is a way of automatically computing the derivatives of the output of a function using the **Chain Rule**   (https://en.wikipedia.org/wiki/Chain_rule ).<br>
Almost every function can be computed as a composition of simple functions which have simple derivatives; consequently, you can compute the derivative of any function that can be written as composition of simpler functions.<br>
Tensorflow uses **Reverse Mode Differentiation**.


#### Here you can find a quick explanation on how backpropagation work https://google-developers.appspot.com/machine-learning/crash-course/backprop-scroll/

TensorFlow provides the tf.GradientTape API that allow to compute the gradient of a computation w.r.t. its input variables.

In [0]:
# Compute the first derivative of function y=x^3 at x=2
x = tf.constant(2.0)

with tf.GradientTape() as g:
    g.watch(x)
    y = #--fill here--#
    
dy_dx = g.gradient(y, x) 
print(dy_dx)

By default, the resources held by a GradientTape are released as soon as GradientTape.gradient() method is called.<br>
To compute multiple gradients over the same computation,you have to create a persistent gradient tape.<br>
This allows multiple calls to the gradient() method as resources are released when the tape object is garbage collected.

In [0]:
# Compute the first and second derivative of function y=x^4 at x=3
x = tf.constant(3.0)

with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x ** 2
    z = y ** 2

dz_dx = #--fill here--# # first derivative
dy_dx = #--fill here--# # second derivative
print(dz_dx,dy_dx)
del g  # Drop the reference to the tape

### Keras
<img src="http://mlclass.epizy.com/lab0_images_notebook/keras.png" width="300px"><br>

Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow.<br>
It allows for easy and fast prototyping and supports both convolutional networks and recurrent networks.<br>
### Most important features:
* User friendliness
* Modularity
* Easy extensibility

### Build a Single Layer Perceptron
Let's build a single layer perceptron composed by one dense layer.

<img src="http://mlclass.epizy.com/lab0_images_notebook/simple_nn.png" width="500px"><br>


In [0]:
def one_dense_layer(x, n_in, n_out):
    # n_in: number of inputs, n_out: number of outputs
    # y = sigmoid(W*x + b)
    # W = [1,1]
    # b = 1
    
    out = #--fill here--#
    return out

In [0]:
x_input = tf.constant([[1,2.]], shape=(1,2))

n_in = 2
n_out = 2

res = # --fill here-- # 
print(res)

### Build the same Single Layer Perceptron with Keras

In [0]:
# Define the number of inputs and outputs
n_input_nodes = 2
n_output_nodes = 2

# First define the model 
model = tf.keras.Sequential() # model lets us define a linear stack of network layers.

# define a single fully connected network layer
# look at https://keras.io/layers/core/ to see which parameters takes as input
dense_layer = #--fill here--#

# Add the dense layer to the model using add() function
# --fill here --#

In [0]:
# Test model
x_input = tf.constant([[1,2.]], shape=(1,2))
print(model(x_input))

In [0]:
# Compare the results obtained
print(tf.reduce_all(tf.equal(model(x_input),one_dense_layer(x_input, n_in=n_in, n_out=n_out))))

### Build a Multilayer perceptron
Let's build a multilayer perceptron; MLPs are fully connected, each node in one layer connects with a certain weight to every node in the following layer.

<img src="http://mlclass.epizy.com/lab0_images_notebook/mlp.png" width="500px"><br>

Try to build one composed by two hidden dense layer with ReLU activation and one dense output layer(units=1) with sigmoid activation.

In [0]:
# Generate dummy data
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

units = 32
input_dim = 100

model = #--fill here--#

# Compile the model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=30, batch_size=32)